# Weather Classification Model Training

Train EfficientNet/ResNet for weather classification


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
import sys

PROJECT_ROOT = Path("/content/drive/MyDrive/NLP")
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
# --- Colab/Drive bootstrap (paste at top of every notebook) ---
from google.colab import drive
drive.mount('/content/drive')                       # mounts Drive interactively

from pathlib import Path
import sys, os, importlib

# 1) Project root on Drive (change only if your folder path is different)
PROJECT_ROOT = Path("/content/drive/MyDrive/NLP").resolve()
print("PROJECT_ROOT:", PROJECT_ROOT)

# 2) Make sure the project root is on sys.path so `import src` and `import config` work
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    print("Inserted PROJECT_ROOT into sys.path")

# 3) Optionally set current working directory to PROJECT_ROOT (so relative writes go to Drive)
os.chdir(str(PROJECT_ROOT))
print("cwd:", os.getcwd())

# 4) Ensure outputs directories exist and point to the expected locations used by notebooks
OUTPUTS = PROJECT_ROOT / "outputs"
MODELS = OUTPUTS / "models"
LOGS = OUTPUTS / "logs"
RESULTS = OUTPUTS / "results"
PROCESSED = OUTPUTS / "processed_data"
for d in (OUTPUTS, MODELS, LOGS, RESULTS, PROCESSED):
    d.mkdir(parents=True, exist_ok=True)
print("Ensured outputs dirs:", [str(p) for p in (OUTPUTS, MODELS, LOGS, RESULTS, PROCESSED)])

# 5) Invalidate import caches and confirm config loads from your project
importlib.invalidate_caches()
import config
print("Loaded config from:", getattr(config, "__file__", None))
# ----------------------------------------------------------------


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PROJECT_ROOT: /content/drive/.shortcut-targets-by-id/1EQfO8nEGZf76dp61vd5hxEgCvUhGfMQE/NLP
Inserted PROJECT_ROOT into sys.path
cwd: /content/drive/.shortcut-targets-by-id/1EQfO8nEGZf76dp61vd5hxEgCvUhGfMQE/NLP
Ensured outputs dirs: ['/content/drive/.shortcut-targets-by-id/1EQfO8nEGZf76dp61vd5hxEgCvUhGfMQE/NLP/outputs', '/content/drive/.shortcut-targets-by-id/1EQfO8nEGZf76dp61vd5hxEgCvUhGfMQE/NLP/outputs/models', '/content/drive/.shortcut-targets-by-id/1EQfO8nEGZf76dp61vd5hxEgCvUhGfMQE/NLP/outputs/logs', '/content/drive/.shortcut-targets-by-id/1EQfO8nEGZf76dp61vd5hxEgCvUhGfMQE/NLP/outputs/results', '/content/drive/.shortcut-targets-by-id/1EQfO8nEGZf76dp61vd5hxEgCvUhGfMQE/NLP/outputs/processed_data']
Loaded config from: /content/drive/.shortcut-targets-by-id/1EQfO8nEGZf76dp61vd5hxEgCvUhGfMQE/NLP/config.py


In [ ]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from tqdm import tqdm

sys.path.append(str(Path.cwd().parent))
import config
from src.models.weather_classifier import create_weather_classifier
from src.data_loader import WeatherDataset
from src.utils import set_seed, get_device, save_json
from src.preprocessing import get_weather_preprocessing_pipeline

set_seed(42)
device = get_device()
print(f"Device: {device}")

# Map weather to routing classes
def map_to_routing(weather_name):
    w = weather_name.lower()
    if w in ['shine', 'sunrise', 'cloudy']: return 0  # clear
    elif w == 'rain': return 1  # rain
    elif w in ['fogsmog', 'fog']: return 2  # fog
    elif w in ['snow', 'frost', 'rime', 'hail']: return 3  # snow
    elif w in ['sandstorm', 'dew', 'glaze']: return 4  # haze
    else: return 5  # other

routing_classes = {}
for w in list(config.WEATHER_CLASSES_6K.keys()) + list(config.WEATHER_CLASSES_1K.keys()):
    routing_classes[w] = map_to_routing(w)

# Create datasets
transform = get_weather_preprocessing_pipeline(224)
ds6k = WeatherDataset(config.WEATHER6K_DIR, routing_classes, transform)
ds1k = WeatherDataset(config.WEATHER1K_DIR, routing_classes, transform)
combined = ConcatDataset([ds6k, ds1k])

# Split
train_size = int(0.7 * len(combined))
val_size = int(0.15 * len(combined))
train_ds, val_ds, test_ds = torch.utils.data.random_split(combined, [train_size, val_size, len(combined)-train_size-val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2)

print(f"Train: {len(train_ds)}, Val: {len(val_ds)}, Test: {len(test_ds)}")


Device: cpu
Train: 5590, Val: 1198, Test: 1199


In [ ]:
# Use this where the training loop or checkpoint callback is set up.
CHECKPOINT_DIR = Path("/content/drive/MyDrive/NLP/outputs/models")
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
print("Checkpoint dir:", CHECKPOINT_DIR)

# Example for PyTorch: ensure your saver/torch.save uses CHECKPOINT_DIR / "name.pth"
# Example for any framework: set model_save_path = str(CHECKPOINT_DIR / "model_best.pth")


Checkpoint dir: /content/drive/MyDrive/NLP/outputs/models


In [ ]:
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import os

path = "/content/drive/MyDrive/NLP/outputs/models"
os.makedirs(path, exist_ok=True)
print("Exists:", os.path.exists(path))
!ls -la "/content/drive/MyDrive/NLP/outputs/models"


Exists: True
total 0
-rw------- 1 root root 0 Nov 19 15:33 test.txt


In [ ]:
from pathlib import Path
MODELS_DIR = Path("/content/drive/MyDrive/NLP/outputs/models")
test_bin = MODELS_DIR / "test_write.bin"
with open(test_bin, "wb") as f:
    f.write(b"ok")
print("Wrote:", test_bin.exists(), test_bin, "size:", test_bin.stat().st_size)


Wrote: True /content/drive/MyDrive/NLP/outputs/models/test_write.bin size: 2


In [ ]:
# Create model
model = create_weather_classifier('efficientnet_b0', 6, pretrained=True).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    loss_sum, correct, total = 0, 0, 0
    for imgs, labels in tqdm(loader, desc="Train"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return loss_sum/len(loader), 100*correct/total

def validate(model, loader, criterion, device):
    model.eval()
    loss_sum, correct, total = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Val"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss_sum += loss.item()
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (pred == labels).sum().item()
    return loss_sum/len(loader), 100*correct/total

# Train
best_val_acc = 0
for epoch in range(20):
    print(f"\nEpoch {epoch+1}/20")
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    scheduler.step(val_loss)
    print(f"Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}% | Val: Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), config.MODELS_DIR / 'weather_classifier_best.pth')
        print(f"Saved best model (val_acc={val_acc:.2f}%)")

# Test
model.load_state_dict(torch.load(config.MODELS_DIR / 'weather_classifier_best.pth'))
test_loss, test_acc = validate(model, test_loader, criterion, device)
print(f"\nTest: Loss={test_loss:.4f}, Acc={test_acc:.2f}%")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]


Epoch 1/20


Val: 100%|██████████| 38/38 [05:45<00:00,  9.10s/it]


Train: Loss=0.8211, Acc=75.01% | Val: Loss=0.2868, Acc=91.40%
Saved best model (val_acc=91.40%)

Epoch 2/20


...

Train: Loss=0.0050, Acc=99.87% | Val: Loss=0.2014, Acc=95.08%

Epoch 20/20


Val: 100%|██████████| 38/38 [01:21<00:00,  2.14s/it]


Train: Loss=0.0045, Acc=99.93% | Val: Loss=0.2038, Acc=95.33%


Val: 100%|██████████| 38/38 [06:12<00:00,  9.80s/it]


Test: Loss=0.2191, Acc=93.33%
